In [7]:
import nltk
import numpy as np
import enchant
import pandas as pd
import os
import re
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')      

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Micha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Micha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Micha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
def Replace_all_numbers(data, replace):
    if type(data) == type([]):
        data = " ".join(data)
        
    data = re.sub('\d+',replace, data)
    
    return data.split(" ")

In [9]:
def Formating(data):
    # Bierzemy wszystko po pierwszych 2 enterach czyli po opisie technicznym
    formated = data.split('\n\n')[1:]
    formated = ' '.join(formated).lower() # zamiana na małe litery
    
    # Wyrzucamy wszystkie wyrażenia słowo.słowo(ścieżki itp.) i adresy mail
    formated = re.sub('((\w+\.)+\w+)|\w+@\w+','', formated)
    
    # Ekstraktujemy słowa z tekstu i tokenizujemy
    tokenizer = RegexpTokenizer(r"\w+")
    formated= tokenizer.tokenize(formated)
    
    # Podmieniamy wszystkie liczby i ciągi liczb na spacje 
    formated = Replace_all_numbers(formated, ' ')
    
    # Usuwamy stop wordsy
    stop_words = set(stopwords.words('english'))
    formated = [word for word in formated if not word in stop_words]
    
    # Usuwamy krótkie słowa do 2 liter i stemmujemy(autokorekta za długo się liczy)
    ps = PorterStemmer()
    formated = [ps.stem(word) for word in formated if len(word) > 2]
    
    # Wybieramy słowa z języka angielskiego
    d = enchant.Dict("en_US")
    formated = [word for word in formated if(word !='' and  d.check(word))]
    
    return formated

In [10]:
data = []
# ścieżka do plików
p = os.getcwd()+'\data'
for folder in os.listdir(p):
    mails = os.listdir(p + '/' + folder)
    for mail in mails:
        path = p + '/' + folder + '/' + mail
        with open(path) as file:
            text = file.read()
        data.append([text, folder])

In [13]:
# Preprocesing
data = [[Formating(text),folder] for text, folder in data]

In [14]:
# Bag of words, trzeba wcześniej jeszcze splita zrobić
matrix = CountVectorizer()
df = list(np.array(data)[:,0])
df = [' '.join(text) for text in df]
X = matrix.fit_transform(df).toarray()

C:\Users\Micha\AppData\Local\Temp\ipykernel_27148\669902624.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df = list(np.array(data)[:,0])


In [15]:
# Do ramki danych 
df = pd.DataFrame(X, columns = matrix.get_feature_names_out())
df['Folder'] = np.array(data)[:,1]
df

C:\Users\Micha\AppData\Local\Temp\ipykernel_27148\530713447.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df['Folder'] = np.array(data)[:,1]


,aa,aah,aardvark,ab,aba,aback,abandon,abba,abbey,abbot,...,zodiac,zonal,zone,zoo,zooid,zoologist,zoom,zoophilia,zucchini,Folder
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,alt.atheism
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,alt.atheism
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,alt.atheism
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,alt.atheism
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,alt.atheism
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,talk.religion.misc
19993,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,talk.religion.misc
19994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,talk.religion.misc
19995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,talk.religion.misc
